In [1]:
import tkinter as tk
from tkinter import messagebox

# Initialize main window
root = tk.Tk()
root.title("Tic-Tac-Toe with AI")

# Enable full screen mode
root.attributes('-fullscreen', True)

# Bind Escape key to exit full screen
def exit_fullscreen(event=None):
    root.attributes('-fullscreen', False)
root.bind("<Escape>", exit_fullscreen)

# Create a Canvas for colorful background
canvas = tk.Canvas(root, width=root.winfo_screenwidth(), height=root.winfo_screenheight())
canvas.pack(fill="both", expand=True)

# Create a vertical gradient background on the canvas
def draw_gradient(canvas, width, height, start_color, end_color):
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    def rgb_to_hex(rgb):
        return '#%02x%02x%02x' % rgb

    start_rgb = hex_to_rgb(start_color)
    end_rgb = hex_to_rgb(end_color)

    steps = height
    for i in range(steps):
        ratio = i / steps
        r = int(start_rgb[0] + (end_rgb[0] - start_rgb[0]) * ratio)
        g = int(start_rgb[1] + (end_rgb[1] - start_rgb[1]) * ratio)
        b = int(start_rgb[2] + (end_rgb[2] - start_rgb[2]) * ratio)
        color = rgb_to_hex((r, g, b))
        canvas.create_line(0, i, width, i, fill=color)

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
draw_gradient(canvas, screen_width, screen_height, "#89CFF0", "#034078")  # Light blue to dark blue

# Create a frame on top of the canvas for widgets (centered and sized)
frame_width = 480
frame_height = 600
frame = tk.Frame(root, bg="#f0f0f0")
frame.place(relx=0.5, rely=0.5, anchor="center", width=frame_width, height=frame_height)

# Game variables
player = "X"  # Human player
board = [["" for _ in range(3)] for _ in range(3)]
buttons = [[None for _ in range(3)] for _ in range(3)]

def check_winner(b=None):
    b = b if b else board
    for i in range(3):
        if b[i][0] == b[i][1] == b[i][2] != "":
            return b[i][0]
        if b[0][i] == b[1][i] == b[2][i] != "":
            return b[0][i]
    if b[0][0] == b[1][1] == b[2][2] != "":
        return b[0][0]
    if b[0][2] == b[1][1] == b[2][0] != "":
        return b[0][2]
    return None

def is_draw(b=None):
    b = b if b else board
    for row in b:
        if "" in row:
            return False
    return True

def minimax(b, depth, is_maximizing):
    winner = check_winner(b)
    if winner == "O":
        return 1
    elif winner == "X":
        return -1
    elif is_draw(b):
        return 0

    if is_maximizing:
        best_score = -float("inf")
        for i in range(3):
            for j in range(3):
                if b[i][j] == "":
                    b[i][j] = "O"
                    score = minimax(b, depth + 1, False)
                    b[i][j] = ""
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = float("inf")
        for i in range(3):
            for j in range(3):
                if b[i][j] == "":
                    b[i][j] = "X"
                    score = minimax(b, depth + 1, True)
                    b[i][j] = ""
                    best_score = min(score, best_score)
        return best_score

def ai_move():
    best_score = -float("inf")
    move = None
    for i in range(3):
        for j in range(3):
            if board[i][j] == "":
                board[i][j] = "O"
                score = minimax(board, 0, False)
                board[i][j] = ""
                if score > best_score:
                    best_score = score
                    move = (i, j)
    if move:
        board[move[0]][move[1]] = "O"
        buttons[move[0]][move[1]].config(text="O", state="disabled", disabledforeground="#d9534f")
        winner = check_winner()
        if winner:
            messagebox.showinfo("Game Over", f"Player {winner} wins!")
            reset_game()
        elif is_draw():
            messagebox.showinfo("Game Over", "It's a Draw!")
            reset_game()
        else:
            global player
            player = "X"
            status_label.config(text="Player X's turn")

def on_click(row, col):
    global player
    if board[row][col] == "" and player == "X":
        board[row][col] = player
        buttons[row][col].config(text=player, state="disabled", disabledforeground="#0275d8")
        winner = check_winner()
        if winner:
            messagebox.showinfo("Game Over", f"Player {winner} wins!")
            reset_game()
        elif is_draw():
            messagebox.showinfo("Game Over", "It's a Draw!")
            reset_game()
        else:
            player = "O"
            status_label.config(text="Player O's (AI) turn")
            root.after(400, ai_move)  # AI plays after short delay

def reset_game():
    global player, board
    player = "X"
    board = [["" for _ in range(3)] for _ in range(3)]
    for i in range(3):
        for j in range(3):
            buttons[i][j].config(text="", state="normal")
    status_label.config(text="Player X's turn")

# UI Setup inside frame

# Title Label
title_label = tk.Label(frame, text="Tic-Tac-Toe", font=("Helvetica", 36, "bold"), bg="#f0f0f0", fg="#333")
title_label.pack(pady=(20, 10))

# Status Label
status_label = tk.Label(frame, text="Player X's turn", font=("Helvetica", 20), bg="#f0f0f0", fg="#555")
status_label.pack(pady=(0, 20))

# Board Frame with thick border and background
board_frame = tk.Frame(frame, bg="#222", bd=6, relief="ridge")
board_frame.pack(pady=10)

# Create larger buttons with thick grid lines
button_size = 120
font_size = 56

for i in range(3):
    for j in range(3):
        btn = tk.Button(
            board_frame, text="", font=("Helvetica", font_size, "bold"),
            width=3, height=1,
            relief="flat",
            bg="white",
            activebackground="#e6e6e6",
            command=lambda r=i, c=j: on_click(r, c)
        )
        btn.grid(row=i, column=j, sticky="nsew")

        # Add padding to simulate grid lines
        if j < 2:
            btn.grid_configure(padx=(0, 8))
        if i < 2:
            btn.grid_configure(pady=(0, 8))

        buttons[i][j] = btn

# Configure grid weights for uniform button size
for i in range(3):
    board_frame.grid_rowconfigure(i, weight=1, minsize=button_size)
    board_frame.grid_columnconfigure(i, weight=1, minsize=button_size)

# Reset Button
reset_btn = tk.Button(frame, text="Reset Game", font=("Helvetica", 20), bg="#0275d8", fg="white",
                      activebackground="#025aa5", activeforeground="white",
                      relief="raised", bd=4, command=reset_game)
reset_btn.pack(pady=30, ipadx=20, ipady=10)

root.mainloop()
